![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)

# 🧠 Section 4: Memory Tools and LangGraph Fundamentals

**⏱️ Estimated Time:** 45-60 minutes

## 🎯 Learning Objectives

By the end of this notebook, you will:

1. **Understand** how memory tools enable active context engineering
2. **Build** the three essential memory tools: store, search, and retrieve
3. **Learn** LangGraph fundamentals (nodes, edges, state)
4. **Compare** passive vs active memory management
5. **Prepare** for building a full course advisor agent

---

## 🔗 Bridge from Previous Sections

### **What You've Learned:**

**Section 1:** Context Types
- System, User, Conversation, Retrieved context
- How context shapes LLM responses

**Section 2:** RAG Foundations
- Semantic search with vector embeddings
- Retrieving relevant information
- Context assembly and generation

**Section 3:** Memory Architecture
- Working memory for conversation continuity
- Long-term memory for persistent knowledge
- Memory-enhanced RAG systems

### **What's Next: Memory Tools for Context Engineering**

**Section 3 Approach:**
- Memory operations hardcoded in your application flow
- You explicitly call `get_working_memory()`, `search_long_term_memory()`, etc.
- Fixed sequence: load → search → generate → save

**Section 4 Approach (This Section):**
- LLM decides when to use memory tools
- LLM chooses what information to store and retrieve
- Dynamic decision-making based on conversation context

**💡 Key Insight:** Memory tools let the LLM actively decide when to use memory, rather than having it hardcoded

---

## 🧠 Memory Tools: The Context Engineering Connection

**Why memory tools matter for context engineering:**

Recall the **four context types** from Section 1:
1. **System Context** (static instructions)
2. **User Context** (profile, preferences) ← **Memory tools help build this**
3. **Conversation Context** (session history) ← **Memory tools help manage this**
4. **Retrieved Context** (RAG results)

**Memory tools enable dynamic context construction:**

### **Section 3 Approach:**
```python
# Hardcoded in application flow
async def memory_enhanced_rag_query(user_query, session_id, student_id):
    working_memory = await memory_client.get_working_memory(...)
    long_term_facts = await memory_client.search_long_term_memory(...)
    # ... fixed sequence of operations
```

### **Section 4 Approach (This Section):**
```python
# LLM decides when to use tools
@tool
def store_memory(text: str):
    """Store important information in long-term memory."""

@tool
def search_memories(query: str):
    """Search long-term memory for relevant facts."""

# LLM calls these tools when it determines they're needed
```

---

## 🔧 The Three Essential Memory Tools

### **1. `store_memory` - Save Important Information**

**When to use:**
- User shares preferences, goals, constraints
- Important facts emerge during conversation
- Context that should persist across sessions

**Example:**
```
User: "I prefer online courses because I work full-time"
Agent: [Thinks: "This is important context I should remember"]
Agent: [Calls: store_memory("User prefers online courses due to full-time work")]
Agent: "I'll remember your preference for online courses..."
```

### **2. `search_memories` - Find Relevant Past Information**

**When to use:**
- Need context about user's history or preferences
- User asks about past conversations
- Building personalized responses

**Example:**
```
User: "What courses should I take next semester?"
Agent: [Thinks: "I need to know their preferences and past courses"]
Agent: [Calls: search_memories("course preferences major interests completed")]
Memory: "User is CS major, interested in AI, prefers online, completed CS101"
Agent: "Based on your CS major and AI interest..."
```

### **3. `retrieve_memories` - Get Specific Stored Facts**

**When to use:**
- Need to recall exact details from past conversations
- User references something specific they mentioned before
- Verifying stored information

**Example:**
```
User: "What was that GPA requirement we discussed?"
Agent: [Calls: retrieve_memories("GPA requirement graduation")]
Memory: "User needs 3.5 GPA for honors program admission"
Agent: "You mentioned needing a 3.5 GPA for the honors program"
```

---

## 📦 Setup and Environment

### ⚠️ **IMPORTANT: Prerequisites Required**

**Before running this notebook, you MUST have:**

1. **Redis running** on port 6379
2. **Agent Memory Server running** on port 8088  
3. **OpenAI API key** configured

**🚀 Quick Setup:**
```bash
# Navigate to notebooks_v2 directory
cd ../../

# Check if services are running
./check_setup.sh

# If services are down, run setup
./setup_memory_server.sh
```

**📖 Detailed Setup:** See `../SETUP_GUIDE.md` for complete instructions.

---


### Automated Setup Check

Let's run the setup script to ensure all services are running properly.


In [1]:
# Run the setup script to ensure Redis and Agent Memory Server are running
import subprocess
import sys
from pathlib import Path

# Path to setup script
setup_script = Path("../../reference-agent/setup_agent_memory_server.py")

if setup_script.exists():
    print("Running automated setup check...\n")
    result = subprocess.run(
        [sys.executable, str(setup_script)],
        capture_output=True,
        text=True
    )
    print(result.stdout)
    if result.returncode != 0:
        print("⚠️  Setup check failed. Please review the output above.")
        print(result.stderr)
    else:
        print("\n✅ All services are ready!")
else:
    print("⚠️  Setup script not found. Please ensure services are running manually.")


Running automated setup check...




🔧 Agent Memory Server Setup
📊 Checking Redis...
✅ Redis is running
📊 Checking Agent Memory Server...
🔍 Agent Memory Server container exists. Checking health...
✅ Agent Memory Server is running and healthy
✅ No Redis connection issues detected

✅ Setup Complete!
📊 Services Status:
   • Redis: Running on port 6379
   • Agent Memory Server: Running on port 8088

🎯 You can now run the notebooks!


✅ All services are ready!


---


### Install Dependencies

If you haven't already installed the reference-agent package, uncomment and run the following:


In [2]:
# Uncomment to install reference-agent package
# %pip install -q -e ../../reference-agent

# Uncomment to install agent-memory-client
# %pip install -q agent-memory-client


### Environment Configuration


In [3]:
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv("../../reference-agent/.env")

# Verify required environment variables
required_vars = {
    "OPENAI_API_KEY": "OpenAI API key for LLM",
    "REDIS_URL": "Redis connection for vector storage",
    "AGENT_MEMORY_URL": "Agent Memory Server for memory tools"
}

missing_vars = []
for var, description in required_vars.items():
    if not os.getenv(var):
        missing_vars.append(f"  - {var}: {description}")

if missing_vars:
    raise ValueError(f"""
    ⚠️  Missing required environment variables:
    
{''.join(missing_vars)}
    
    Please create a .env file in the reference-agent directory:
    1. cd ../../reference-agent
    2. cp .env.example .env
    3. Edit .env and add your API keys
    """)

print("✅ Environment configured successfully!")
print(f"   OpenAI Model: {os.getenv('OPENAI_MODEL', 'gpt-4o')}")
print(f"   Redis URL: {os.getenv('REDIS_URL', 'redis://localhost:6379')}")
print(f"   Memory Server: {os.getenv('AGENT_MEMORY_URL', 'http://localhost:8088')}")

✅ Environment configured successfully!
   OpenAI Model: gpt-4o
   Redis URL: redis://localhost:6379
   Memory Server: http://localhost:8088


### Service Health Check

Before building memory tools, let's verify that Redis and the Agent Memory Server are running.


In [4]:
import requests
import redis

def check_redis():
    """Check if Redis is accessible."""
    try:
        r = redis.from_url(os.getenv("REDIS_URL", "redis://localhost:6379"))
        r.ping()
        return True, "Connected successfully"
    except Exception as e:
        return False, str(e)

def check_memory_server():
    """Check if Agent Memory Server is accessible."""
    try:
        url = os.getenv("AGENT_MEMORY_URL", "http://localhost:8088")
        response = requests.get(f"{url}/v1/health", timeout=5)
        return response.status_code == 200, f"Status: {response.status_code}"
    except Exception as e:
        return False, str(e)

# Check services
print("🔍 Checking required services...\n")

redis_ok, redis_msg = check_redis()
print(f"Redis: {'✅' if redis_ok else '❌'} {redis_msg}")

memory_ok, memory_msg = check_memory_server()
print(f"Agent Memory Server: {'✅' if memory_ok else '❌'} {memory_msg}")

if not (redis_ok and memory_ok):
    print("\n⚠️  Some services are not running. Please start them:")
    if not redis_ok:
        print("   Redis: docker run -d -p 6379:6379 redis/redis-stack:latest")
    if not memory_ok:
        print("   Memory Server: cd ../../reference-agent && python setup_agent_memory_server.py")
else:
    print("\n✅ All services are running!")

🔍 Checking required services...

Redis: ✅ Connected successfully
Agent Memory Server: ✅ Status: 200

✅ All services are running!


---

## 🛠️ Building Memory Tools

Now let's build the three essential memory tools. We'll start simple and build up complexity.

### **Step 1: Initialize Memory Client**


In [5]:
from agent_memory_client import MemoryAPIClient, MemoryClientConfig
from agent_memory_client.models import ClientMemoryRecord
from agent_memory_client.filters import UserId

# Initialize memory client
config = MemoryClientConfig(
    base_url=os.getenv("AGENT_MEMORY_URL", "http://localhost:8088"),
    default_namespace="redis_university"
)
memory_client = MemoryAPIClient(config=config)

# Test user for this notebook
test_user_id = "student_memory_tools_demo"
test_session_id = "session_memory_tools_demo"

print(f"✅ Memory client initialized")
print(f"   Base URL: {config.base_url}")
print(f"   Namespace: {config.default_namespace}")
print(f"   Test User: {test_user_id}")

✅ Memory client initialized
   Base URL: http://localhost:8088
   Namespace: redis_university
   Test User: student_memory_tools_demo


---

## 🛠️ Understanding Tools in LLM Applications

### **What Are Tools?**

**Tools** are functions that LLMs can call to interact with external systems, retrieve information, or perform actions beyond text generation.

**Think of tools as:**
- 🔌 **Extensions** to the LLM's capabilities
- 🤝 **Interfaces** between the LLM and external systems
- 🎯 **Actions** the LLM can take to accomplish tasks

### **How Tool Calling Works**

```
1. User Input → "Store my preference for online courses"
                 ↓
2. LLM Analysis → Decides: "I need to use store_memory tool"
                 ↓
3. Tool Call → Returns structured function call with arguments
                 ↓
4. Tool Execution → Your code executes the function
                 ↓
5. Tool Result → Returns result to LLM
                 ↓
6. LLM Response → Generates final text response using tool result
```

### **Tool Definition Components**

Every tool needs three key components:

**1. Input Schema (Pydantic Model)**
```python
class StoreMemoryInput(BaseModel):
    text: str = Field(description="What to store")
    memory_type: str = Field(default="semantic")
    topics: List[str] = Field(default=[])
```
- Defines what parameters the tool accepts
- Provides descriptions that help the LLM understand usage
- Validates input types

**2. Tool Function**
```python
@tool("store_memory", args_schema=StoreMemoryInput)
async def store_memory(text: str, memory_type: str = "semantic", topics: List[str] = None) -> str:
    # Implementation
    return "Success message"
```
- The actual function that performs the action
- Must return a string (the LLM reads this result)
- Can be sync or async

**3. Docstring (Critical!)**
```python
"""
Store important information in long-term memory.

Use this tool when:
- User shares preferences, goals, or constraints
- Important facts emerge during conversation

Examples:
- "User prefers online courses"
- "User is CS major interested in AI"
"""
```
- The LLM reads this to decide when to use the tool
- Should include clear use cases and examples
- More detailed = better tool selection

### **Best Practices for Tool Design**

#### **1. Clear, Descriptive Names**
```python
✅ Good: store_memory, search_courses, get_user_profile
❌ Bad: do_thing, process, handle_data
```

#### **2. Detailed Descriptions**
```python
✅ Good: "Store important user preferences and facts in long-term memory for future conversations"
❌ Bad: "Stores data"
```

#### **3. Specific Use Cases in Docstring**
```python
✅ Good:
"""
Use this tool when:
- User explicitly shares preferences
- Important facts emerge that should persist
- Information will be useful for future recommendations
"""

❌ Bad:
"""
Stores information.
"""
```

#### **4. Return Meaningful Results**
```python
✅ Good: return f"Stored: {text} with topics {topics}"
❌ Bad: return "Done"
```
The LLM uses the return value to understand what happened and craft its response.

#### **5. Handle Errors Gracefully**
```python
✅ Good:
try:
    result = await memory_client.create_long_term_memory([record])
    return f"Successfully stored: {text}"
except Exception as e:
    return f"Could not store memory: {str(e)}"
```
Always return a string explaining what went wrong.

#### **6. Keep Tools Focused**
```python
✅ Good: Separate tools for store_memory, search_memories, retrieve_memories
❌ Bad: One generic memory_operation(action, data) tool
```
Focused tools are easier for LLMs to select correctly.

### **Common Tool Patterns**

**Information Retrieval:**
- Search databases
- Query APIs
- Fetch user data

**Data Storage:**
- Save preferences
- Store conversation facts
- Update user profiles

**External Actions:**
- Send emails
- Create calendar events
- Make API calls

**Computation:**
- Calculate values
- Process data
- Generate reports

---

### **Step 2: Build the `store_memory` Tool**

Now let's build our first memory tool following these best practices.


In [6]:
from langchain_core.tools import tool
from pydantic import BaseModel, Field
from typing import List, Optional

class StoreMemoryInput(BaseModel):
    """Input schema for storing memories."""
    text: str = Field(
        description="The information to store. Should be clear, specific, and important for future conversations."
    )
    memory_type: str = Field(
        default="semantic",
        description="Type of memory: 'semantic' for facts/preferences, 'episodic' for events/experiences"
    )
    topics: List[str] = Field(
        default=[],
        description="List of topics/tags for this memory (e.g., ['preferences', 'courses', 'career'])"
    )

@tool("store_memory", args_schema=StoreMemoryInput)
async def store_memory(text: str, memory_type: str = "semantic", topics: List[str] = None) -> str:
    """
    Store important information in long-term memory.
    
    Use this tool when:
    - User shares preferences, goals, or constraints
    - Important facts emerge during conversation
    - Information should persist across sessions
    - Context that will be useful for future recommendations
    
    Examples:
    - "User prefers online courses due to work schedule"
    - "User is Computer Science major interested in AI"
    - "User completed CS101 with grade A"
    
    Returns: Confirmation that memory was stored
    """
    try:
        # Create memory record
        memory_record = ClientMemoryRecord(
            text=text,
            memory_type=memory_type,
            topics=topics or [],
            user_id=test_user_id
        )
        
        # Store in long-term memory
        await memory_client.create_long_term_memory([memory_record])
        
        return f"Stored: {text}"
    except Exception as e:
        return f"Error storing memory: {str(e)}"

# Test the tool
test_result = await store_memory.ainvoke({
    "text": "User prefers online courses for testing",
    "memory_type": "semantic",
    "topics": ["preferences", "test"]
})
print(f"🧠 Store Memory Test: {test_result}")

🧠 Store Memory Test: Stored: User prefers online courses for testing


### **Step 3: Build the `search_memories` Tool**

This tool allows the LLM to search its long-term memory for relevant information.


In [7]:
class SearchMemoriesInput(BaseModel):
    """Input schema for searching memories."""
    query: str = Field(
        description="Search query to find relevant memories. Use keywords related to what you need to know."
    )
    limit: int = Field(
        default=5,
        description="Maximum number of memories to return. Default is 5."
    )

@tool("search_memories", args_schema=SearchMemoriesInput)
async def search_memories(query: str, limit: int = 5) -> str:
    """
    Search long-term memory for relevant information.
    
    Use this tool when:
    - Need context about user's preferences or history
    - User asks about past conversations
    - Building personalized responses
    - Need to recall what you know about the user
    
    Examples:
    - query="course preferences" → finds preferred course types
    - query="completed courses" → finds courses user has taken
    - query="career goals" → finds user's career interests
    
    Returns: Relevant memories or "No memories found"
    """
    try:
        # Search long-term memory
        results = await memory_client.search_long_term_memory(
            text=query,
            user_id=UserId(eq=test_user_id),
            limit=limit
        )

        if not results or not results.memories:
            return "No memories found matching your query."

        # Format results
        memory_texts = []
        for memory in results.memories:
            memory_texts.append(f"- {memory.text}")

        return "\n".join(memory_texts)
    except Exception as e:
        return f"Error searching memories: {str(e)}"

# Test the tool
test_result = await search_memories.ainvoke({
    "query": "preferences",
    "limit": 5
})
print(f"🔍 Search Memories Test: {test_result}")

🔍 Search Memories Test: - User prefers online courses for testing
- User is a Computer Science major interested in AI and machine learning. Prefers online courses due to part-time work.


### **Step 4: Build the `retrieve_memories` Tool**

This tool allows the LLM to retrieve specific stored facts.


In [8]:
class RetrieveMemoriesInput(BaseModel):
    """Input schema for retrieving specific memories."""
    topics: List[str] = Field(
        description="List of specific topics to retrieve (e.g., ['GPA', 'requirements', 'graduation'])"
    )
    limit: int = Field(
        default=3,
        description="Maximum number of memories to return. Default is 3."
    )

@tool("retrieve_memories", args_schema=RetrieveMemoriesInput)
async def retrieve_memories(topics: List[str], limit: int = 3) -> str:
    """
    Retrieve specific stored facts by topic.
    
    Use this tool when:
    - Need to recall exact details from past conversations
    - User references something specific they mentioned before
    - Verifying stored information
    - Looking for facts about specific topics
    
    Examples:
    - topics=["GPA", "requirements"] → finds GPA-related memories
    - topics=["completed", "courses"] → finds completed course records
    - topics=["career", "goals"] → finds career-related memories
    
    Returns: Specific memories matching the topics
    """
    try:
        # Search for memories with specific topics
        query = " ".join(topics)
        results = await memory_client.search_long_term_memory(
            text=query,
            user_id=UserId(eq=test_user_id),
            limit=limit
        )

        if not results or not results.memories:
            return f"No memories found for topics: {', '.join(topics)}"

        # Format results with topics
        memory_texts = []
        for memory in results.memories:
            topics_str = ", ".join(memory.topics) if memory.topics else "general"
            memory_texts.append(f"[{topics_str}] {memory.text}")

        return "\n".join(memory_texts)
    except Exception as e:
        return f"Error retrieving memories: {str(e)}"

# Test the tool
test_result = await retrieve_memories.ainvoke({
    "topics": ["preferences", "test"],
    "limit": 3
})
print(f"📋 Retrieve Memories Test: {test_result}")

📋 Retrieve Memories Test: [preferences, test] User prefers online courses for testing
[preferences, academic, career] User is a Computer Science major interested in AI and machine learning. Prefers online courses due to part-time work.


### **Step 5: Test Memory Tools with LLM**

Now let's see how an LLM uses these memory tools.


In [9]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage

# Initialize LLM with memory tools
llm = ChatOpenAI(model=os.getenv("OPENAI_MODEL", "gpt-4o"), temperature=0)
memory_tools = [store_memory, search_memories, retrieve_memories]
llm_with_tools = llm.bind_tools(memory_tools)

# System message for memory-aware agent
system_prompt = """
You are a Redis University course advisor with memory tools.

IMPORTANT: Use your memory tools strategically:
- When users share preferences, goals, or important facts → use store_memory
- When you need context about the user → use search_memories
- When users reference specific past information → use retrieve_memories

Always explain what you're doing with memory to help users understand.
"""

# Test conversation
messages = [
    SystemMessage(content=system_prompt),
    HumanMessage(content="Hi! I'm a Computer Science major interested in AI and machine learning. I prefer online courses because I work part-time.")
]

response = llm_with_tools.invoke(messages)
print("🤖 LLM Response:")
print(f"   Tool calls: {len(response.tool_calls) if response.tool_calls else 0}")
if response.tool_calls:
    for i, tool_call in enumerate(response.tool_calls):
        print(f"   Tool {i+1}: {tool_call['name']}")
        print(f"   Args: {tool_call['args']}")
print(f"\n💬 Response: {response.content}")

# Explain the empty response
if response.tool_calls and not response.content:
    print("\n📝 Note: The response is empty because the LLM decided to call a tool instead of")
    print("   generating text. This is expected behavior! The LLM is saying:")
    print("   'I need to store this information first, then I'll respond.'")
    print("\n   To get the final response, we would need to:")
    print("   1. Execute the tool call (store_memory)")
    print("   2. Send the tool result back to the LLM")
    print("   3. Get the LLM's final text response")
    print("\n   This multi-step process is exactly why we need LangGraph! 👇")

🤖 LLM Response:
   Tool calls: 1
   Tool 1: store_memory
   Args: {'text': 'User is a Computer Science major interested in AI and machine learning. Prefers online courses due to part-time work.', 'memory_type': 'semantic', 'topics': ['preferences', 'academic', 'career']}

💬 Response: 

📝 Note: The response is empty because the LLM decided to call a tool instead of
   generating text. This is expected behavior! The LLM is saying:
   'I need to store this information first, then I'll respond.'

   To get the final response, we would need to:
   1. Execute the tool call (store_memory)
   2. Send the tool result back to the LLM
   3. Get the LLM's final text response

   This multi-step process is exactly why we need LangGraph! 👇


---

## 🔄 Complete Tool Execution Loop Example

Let's manually complete the tool execution loop to see the full workflow. This will help you understand what LangGraph automates.


In [10]:
from langchain_core.messages import ToolMessage

print("=" * 80)
print("COMPLETE TOOL EXECUTION LOOP - Manual Implementation")
print("=" * 80)

# Step 1: User input
user_message = "Hi! I'm a Computer Science major interested in AI and machine learning. I prefer online courses because I work part-time."
print(f"\n👤 USER INPUT:\n{user_message}")

# Step 2: LLM decides to use tool
messages = [
    SystemMessage(content=system_prompt),
    HumanMessage(content=user_message)
]

print("\n" + "=" * 80)
print("STEP 1: LLM Analysis")
print("=" * 80)
response_1 = llm_with_tools.invoke(messages)
print(f"✅ LLM decided to call: {response_1.tool_calls[0]['name']}")
print(f"   Arguments: {response_1.tool_calls[0]['args']}")

# Step 3: Execute the tool
print("\n" + "=" * 80)
print("STEP 2: Tool Execution")
print("=" * 80)
tool_call = response_1.tool_calls[0]
tool_result = await store_memory.ainvoke(tool_call['args'])
print(f"✅ Tool executed successfully")
print(f"   Result: {tool_result}")

# Step 4: Send tool result back to LLM
print("\n" + "=" * 80)
print("STEP 3: LLM Generates Final Response")
print("=" * 80)
messages.append(response_1)  # Add the tool call message
messages.append(ToolMessage(content=tool_result, tool_call_id=tool_call['id']))  # Add tool result

response_2 = llm_with_tools.invoke(messages)
print(f"✅ Final response generated")
print(f"\n🤖 AGENT RESPONSE:\n{response_2.content}")

# Step 5: Verify memory was stored
print("\n" + "=" * 80)
print("STEP 4: Verify Memory Storage")
print("=" * 80)
search_result = await search_memories.ainvoke({"query": "preferences", "limit": 3})
print(f"✅ Memory verification:")
print(f"{search_result}")

print("\n" + "=" * 80)
print("COMPLETE! This is what LangGraph automates for you.")
print("=" * 80)

COMPLETE TOOL EXECUTION LOOP - Manual Implementation

👤 USER INPUT:
Hi! I'm a Computer Science major interested in AI and machine learning. I prefer online courses because I work part-time.

STEP 1: LLM Analysis


✅ LLM decided to call: store_memory
   Arguments: {'text': 'User is a Computer Science major interested in AI and machine learning. Prefers online courses due to part-time work.', 'memory_type': 'semantic', 'topics': ['preferences', 'academic', 'career']}

STEP 2: Tool Execution
✅ Tool executed successfully
   Result: Stored: User is a Computer Science major interested in AI and machine learning. Prefers online courses due to part-time work.

STEP 3: LLM Generates Final Response


✅ Final response generated

🤖 AGENT RESPONSE:
Great! I've noted that you're a Computer Science major interested in AI and machine learning, and you prefer online courses because you work part-time. If you have any specific questions or need recommendations, feel free to ask!

STEP 4: Verify Memory Storage
✅ Memory verification:
- User prefers online courses for testing
- User is a Computer Science major interested in AI and machine learning. Prefers online courses due to part-time work.

COMPLETE! This is what LangGraph automates for you.


### **Key Takeaways from Manual Loop**

**What we just did manually:**

1. ✅ **Sent user input to LLM** → Got tool call decision
2. ✅ **Executed the tool** → Got result
3. ✅ **Sent result back to LLM** → Got final response
4. ✅ **Verified the action** → Confirmed memory stored

**Why this is tedious:**
- 🔴 Multiple manual steps
- 🔴 Need to track message history
- 🔴 Handle tool call IDs
- 🔴 Manage state between calls
- 🔴 Complex error handling

**What LangGraph does:**
- ✅ Automates all these steps
- ✅ Manages state automatically
- ✅ Handles tool execution loop
- ✅ Provides clear workflow visualization
- ✅ Makes it easy to add more tools and logic

**Now you understand why we need LangGraph!** 👇


---

## 🎨 Introduction to LangGraph

Memory tools are powerful, but managing complex workflows manually gets complicated. **LangGraph** automates this process.

### **What is LangGraph?**

**LangGraph** is a framework for building stateful, multi-step agent workflows using graphs.

### **Core Concepts**

**1. State** - Shared data structure passed between nodes
- Contains messages, context, and intermediate results
- Automatically managed and updated

**2. Nodes** - Functions that process state
- Examples: call LLM, execute tools, format responses
- Each node receives state and returns updated state

**3. Edges** - Connections between nodes
- Can be conditional (if/else logic)
- Determine workflow flow

**4. Graph** - Complete workflow from start to end
- Orchestrates the entire agent process

### **Simple Memory-Enhanced Graph**

```
START
  ↓
[Load Memory] ← Get user context
  ↓
[Agent Node] ← Decides what to do
  ↓
  ├─→ [Memory Tools] ← store/search/retrieve
  │      ↓
  │   [Agent Node] ← Processes memory results
  │
  └─→ [Respond] ← Generates final response
         ↓
[Save Memory] ← Update conversation history
         ↓
        END
```

### **Why LangGraph for Memory Tools?**

**Without LangGraph:**
- Manual tool execution and state management
- Complex conditional logic
- Hard to visualize workflow
- Difficult to add new steps

**With LangGraph:**
- ✅ Automatic tool execution
- ✅ Clear workflow visualization
- ✅ Easy to modify and extend
- ✅ Built-in state management
- ✅ Memory persistence across turns

---

## 🔄 Passive vs Active Memory: The Key Difference

Let's compare the two approaches to understand why memory tools matter.


### **Passive Memory (Section 3)**

**How it works:**
- System automatically saves all conversations
- System automatically extracts facts
- LLM receives memory but can't control it

**Example conversation:**
```
User: "I'm interested in machine learning"
Agent: "Great! Here are some ML courses..." 
System: [Automatically saves: "User interested in ML"]
```

**Pros:**
- ✅ Simple to implement
- ✅ No additional LLM calls
- ✅ Consistent memory storage

**Cons:**
- ❌ LLM can't decide what's important
- ❌ No strategic memory management
- ❌ Can't search memories on demand


### **Active Memory (This Section)**

**How it works:**
- LLM decides what to store
- LLM decides when to search memories
- LLM controls its own context construction

**Example conversation:**
```
User: "I'm interested in machine learning"
Agent: [Thinks: "This is important, I should remember this"]
Agent: [Calls: store_memory("User interested in machine learning")]
Agent: "I'll remember your interest in ML. Here are some courses..."
```

**Pros:**
- ✅ Strategic memory management
- ✅ LLM controls what's important
- ✅ On-demand memory search
- ✅ Better context engineering

**Cons:**
- ❌ More complex to implement
- ❌ Additional LLM calls (cost)
- ❌ Requires careful tool design


### **When to Use Each Approach**

**Use Passive Memory when:**
- Simple applications with predictable patterns
- Cost is a primary concern
- Memory needs are straightforward
- You want automatic memory management

**Use Active Memory when:**
- Complex applications requiring strategic memory
- LLM needs to control its own context
- Dynamic memory management is important
- Building sophisticated agents

**💡 Key Insight:** Active memory tools enable **intelligent context engineering** where the LLM becomes an active participant in managing its own knowledge.

---

## 🎯 Summary and Next Steps

### **What You've Learned**

**Memory Tools for Context Engineering:**
- `store_memory` - Save important information strategically
- `search_memories` - Find relevant context on demand
- `retrieve_memories` - Get specific facts by topic

**LangGraph Fundamentals:**
- State management for complex workflows
- Nodes and edges for agent orchestration
- Automatic tool execution and state updates

**Active vs Passive Memory:**
- Passive: System controls memory automatically
- Active: LLM controls its own memory strategically

### **Context Engineering Connection**

Memory tools transform the **four context types**:

| Context Type | Section 3 (Passive) | Section 4 (Active) |
|-------------|---------------------|--------------------|
| **System** | Static prompt | Static prompt |
| **User** | Auto-extracted profile | LLM builds profile with `store_memory` |
| **Conversation** | Auto-saved history | LLM manages with `search_memories` |
| **Retrieved** | RAG search | Memory-enhanced RAG queries |

### **Next: Building a Complete Agent**

In **Notebook 2**, you'll combine everything:
- ✅ Memory tools (this notebook)
- ✅ Course search tools
- ✅ LangGraph orchestration
- ✅ Redis Agent Memory Server

**Result:** A complete Redis University Course Advisor Agent that actively manages its own memory and context.

---

## 📚 Additional Resources

### **Memory Tools & Context Engineering**
- [Redis Agent Memory Server](https://github.com/redis/agent-memory-server) - Memory persistence
- [Agent Memory Client](https://pypi.org/project/agent-memory-client/) - Python client library

### **LangGraph & Tool Calling**
- [LangGraph Documentation](https://langchain-ai.github.io/langgraph/) - Official docs
- [LangChain Tools](https://python.langchain.com/docs/modules/tools/) - Tool creation guide

### **Context Engineering Concepts**
- Review **Section 1** for context types fundamentals (System, User, Conversation, Retrieved)
- Review **Section 2** for RAG foundations (semantic search, vector embeddings, retrieval)
- Review **Section 3** for passive memory patterns (working memory, long-term memory, automatic extraction)
- Continue to **Section 4 Notebook 2** for complete agent implementation with all concepts integrated

### **Academic Papers**
- [ReAct: Synergizing Reasoning and Acting in Language Models](https://arxiv.org/abs/2210.03629) - Reasoning + acting pattern
- [Toolformer: Language Models Can Teach Themselves to Use Tools](https://arxiv.org/abs/2302.04761) - Tool learning
- [MemGPT: Towards LLMs as Operating Systems](https://arxiv.org/abs/2310.08560) - Memory management for LLMs
- [Retrieval-Augmented Generation](https://arxiv.org/abs/2005.11401) - RAG foundations

### **Agent Design Patterns**
- [Anthropic's Guide to Building Effective Agents](https://www.anthropic.com/research/building-effective-agents) - Best practices
- [LangChain Agent Patterns](https://python.langchain.com/docs/modules/agents/) - Different agent architectures
- [OpenAI Function Calling Guide](https://platform.openai.com/docs/guides/function-calling) - Tool calling fundamentals

### **Production Resources**
- [LangChain Production Guide](https://python.langchain.com/docs/guides/productionization/) - Deploying agents
- [Redis Best Practices](https://redis.io/docs/manual/patterns/) - Production Redis patterns
- [Agent Memory Client](https://pypi.org/project/agent-memory-client/) - Python client for Agent Memory Server